In [85]:
import numpy as np
import pandas as pd
import os

In [86]:
dtype_dict = {
    'STARTTIMEMS': int,
    'ENDTIMEMS': int
}

df = pd.read_csv('./data/2023.csv')#.dropna().astype(dtype_dict)
df.head()

,STARTSTATIONNAME,STARTSTATIONARRONDISSEMENT,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE,ENDSTATIONNAME,ENDSTATIONARRONDISSEMENT,ENDSTATIONLATITUDE,ENDSTATIONLONGITUDE,STARTTIMEMS,ENDTIMEMS
0,Métro Mont-Royal (Utilités publiques / Rivard),Le Plateau-Mont-Royal,45.524236,-73.581552,Chabot / Mont-Royal,Le Plateau-Mont-Royal,45.534134,-73.573524,1698266696468,1.698267e+12
1,Rielle / Wellington,Verdun,45.460156,-73.567001,St-Jacques / McGill,Ville-Marie,45.501441,-73.560144,1698270535502,1.698272e+12
2,Ste-Catherine / Drummond,Ville-Marie,45.498588,-73.574278,Peel / Ottawa,Le Sud-Ouest,45.494200,-73.559985,1698285705974,1.698286e+12
3,Boyer / du Mont-Royal,Le Plateau-Mont-Royal,45.527432,-73.579917,de l'Hôtel-de-Ville / Rachel,Le Plateau-Mont-Royal,45.519897,-73.580106,1698269489932,1.698270e+12
4,Clark / Ontario,Ville-Marie,45.510625,-73.566903,Wolfe / Robin,Ville-Marie,45.519581,-73.560116,1698270006571,1.698270e+12


Extract and format nodes

In [87]:
df_start = df[['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE']].rename(columns={'STARTSTATIONNAME': 'name',
                                                                                                     'STARTSTATIONLATITUDE': 'latitude:float',
                                                                                                     'STARTSTATIONLONGITUDE': 'longitude:float'})
df_end = df[['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE']].rename(columns={'ENDSTATIONNAME': 'name',
                                                                                             'ENDSTATIONLATITUDE': 'latitude:float',
                                                                                             'ENDSTATIONLONGITUDE': 'longitude:float'})
df_start.head()

,name,latitude:float,longitude:float
0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552
1,Rielle / Wellington,45.460156,-73.567001
2,Ste-Catherine / Drummond,45.498588,-73.574278
3,Boyer / du Mont-Royal,45.527432,-73.579917
4,Clark / Ontario,45.510625,-73.566903


In [88]:
nodes = pd.concat([df_start, df_end], ignore_index=True).drop_duplicates()#.dropna()

print(nodes.shape)

(940, 3)


In [89]:
nodes["movieId:ID"] = nodes.index
nodes[":LABEL"] = "Station"
nodes = nodes[["movieId:ID", "name", "latitude:float", "longitude:float", ":LABEL"]]

In [90]:
nodes.head()

,movieId:ID,name,latitude:float,longitude:float,:LABEL
0,0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552,Station
1,1,Rielle / Wellington,45.460156,-73.567001,Station
2,2,Ste-Catherine / Drummond,45.498588,-73.574278,Station
3,3,Boyer / du Mont-Royal,45.527432,-73.579917,Station
4,4,Clark / Ontario,45.510625,-73.566903,Station


In [91]:
nodes['movieId:ID'] = nodes['movieId:ID'].astype(int)
nodes.to_csv('./data/2023_nodes.csv', index=False)

Extract and format edges

In [102]:
# nodes = nodes.drop(columns=[':LABEL'])
# nodes.head()

edges = pd.merge(df, nodes,
                 left_on=['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE'],
                 right_on=['name', 'latitude:float', 'longitude:float'], how='left')

edges = edges[['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE', 'STARTTIMEMS', 'ENDTIMEMS', 'movieId:ID']].rename(columns={'movieId:ID': ":START_ID"})

edges = pd.merge(edges, nodes,
                 left_on=['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE'], 
                 right_on=['name', 'latitude:float', 'longitude:float'], how='left')
# edges.head()
edges = edges[[':START_ID', 'STARTTIMEMS', 'ENDTIMEMS', 'movieId:ID']].rename(columns={'movieId:ID': ":END_ID", 'STARTTIMEMS':'starttimems:int', 'ENDTIMEMS':'endtimems:int'})

edges = edges.dropna()
edges[":TYPE"] = "CYCLES_TO"
edges.head()

,:START_ID,starttimems:int,endtimems:int,:END_ID,:TYPE
0,0,1698266696468,1.698267e+12,509,CYCLES_TO
1,1,1698270535502,1.698272e+12,301,CYCLES_TO
2,2,1698285705974,1.698286e+12,471,CYCLES_TO
3,3,1698269489932,1.698270e+12,424,CYCLES_TO
4,4,1698270006571,1.698270e+12,1641,CYCLES_TO


In [93]:
print(edges.dtypes)

:START_ID            int32
starttimems:int      int64
endtimems:int      float64
:END_ID              int32
:TYPE               object
dtype: object


In [94]:
# dtype_mapping = {':START_ID': int, 'starttimems':int, 'endtimems':int, ':END_ID':int, ':TYPE':str}
# edges = pd.DataFrame(edges, dtype=dtype_mapping)
edges[':START_ID'] = edges[':START_ID'].astype('int64')
edges[':END_ID'] = edges[':END_ID'].astype('int64')
edges['starttimems:int'] = edges['starttimems:int'].astype('int64')
edges['endtimems:int'] = edges['endtimems:int'].astype('int64')
edges.to_csv('./data/2023_edges.csv', index=False)

In [104]:

edges.head()

,:START_ID,starttimems:int,endtimems:int,:END_ID,:TYPE,test,test2
0,0,1698266696468,1.698267e+12,509,CYCLES_TO,1698266696468,1698267092113
1,1,1698270535502,1.698272e+12,301,CYCLES_TO,1698270535502,1698271967951
2,2,1698285705974,1.698286e+12,471,CYCLES_TO,1698285705974,1698285992418
3,3,1698269489932,1.698270e+12,424,CYCLES_TO,1698269489932,1698269814383
4,4,1698270006571,1.698270e+12,1641,CYCLES_TO,1698270006571,1698270440298


In [ ]:
print(df.shape, "vs.", edges.shape)

(11790840, 10) vs. (11730062, 5)
